In [ ]:
import os
import json
import numpy as np
import pandas as pd
import sys

sys.path.append("./scripts/particles/")

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

In [ ]:
import data_handler as dh
import metrics
import utils

In [ ]:
weather_df = pd.read_csv('../storage/particle/weather.csv', index_col='DATE', parse_dates=True)[['TEMPERATURE', 'WIND_DEG', 'WIND_SPEED', 'HUMIDITY']]
weather_df['WIND_DEG'] = np.sin(weather_df['WIND_DEG'].values * np.pi / 180)

df_org = dh.load_data("../storage/particle/data.csv")
df_org = dh.add_pm_diff(df_org)

In [ ]:
df[(df.index >= pd.to_datetime('2022-05-07 09:40')) & (df.index <= pd.to_datetime('2022-09-01'))].to_csv('data.csv', index_label='DATE')

In [ ]:
ll = os.listdir('../projects/particle/kt/bo/conv_01/')
ll = [x for x in ll if len(x.split('_')) != 0 and x.split('_')[0] == 'trial']

In [ ]:
root = '../projects/particle/kt/bo/conv_01/'
dd = []
for l in ll:
    dir_path = root + l
    f = open(dir_path+'/trial.json', 'r')
    dd.append(json.load(f))
    f.close()

In [ ]:
score = [x['score'] for x in dd if x['score'] != None]
params = [x['hyperparameters'] for x in dd if x['score'] != None]

In [ ]:
score_data = np.array(score)

In [ ]:
models = ['conv_09', 'conv_10', 'conv_11', 'conv_14', 'conv_17', 'conv_18', 'conv_19', 'conv_20', 'conv_21', 'conv_22', 'conv_23', 'conv_24']
pms = ['pm1', 'pm2.5', 'pm10']

In [ ]:
pm_dict = {
    'pm1': [],
    'pm2.5': [],
    'pm10': []
}

for m in models:
    for pm in pms:
        pm_dict[pm].append(pd.read_csv(f'../projects/particle/model/{m}/result/metric/result_{pm}.csv', index_col='Metric'))

In [ ]:
data = np.zeros((len(models), 4))
dfs = pm_dict['pm1']
for i, df in enumerate(dfs):
    for j, c in enumerate(list(df.columns)):
        data[i, j] = df[c]['R Square']

In [ ]:
dd = pd.DataFrame(data, index=models, columns=df.columns)
ax = dd.plot(kind='line', marker='o', linestyle='dashed', figsize=(22, 10), fontsize=14)
ax.set_xticks(ticks=np.arange(0, len(models), 1), labels=dd.index, rotation=45)
ax.set_ylabel('$R^2$', fontsize=17)
ax.set_xlabel('Model', fontsize=17)